In [1]:
from Group_X_Module import function1, function2

In [2]:
import pandas as pd
import numpy as np

In [3]:
from sklearn import calibration

# Report Name


# Section 1

The average of my numbers turned out to be

In [4]:
function1(1, 2, 3)

6

# Section 2

## The numbers were all integers:
- this shows that the trends are constant
- this suggests more work should be done
## The chart shows an upward trend

# Section 3

THis is my explanation of the following analysis:
    - bullet point one
    - bullet point two
    
Then, I continue with this analysis:

In [5]:
x = 3 + 4 /2
print(x)

5.0


### Test of my custom function
In this section, I will test if the function works. The result is:

In [6]:
function2(var1=4, var2=5, var3=-10)

-0.3333333333333333